In [105]:
import pandas as pd
import numpy as np
# from datetime import datetime
# import json
from urllib.request import urlopen
import tgfr

In [2]:
# Adjust to/from date
from_param = 'from=2022-04-01T00:00:00.000Z'
to_param = 'to=2022-05-31T23:59:59.000Z'

# Create DataFrame
## Fuel Report
-   **denorm_df** : denom data between concerned period
-   **fuel_report_df** : Create from expand column in fuel report from denorm_df

In [3]:
import tgfr
denorm_df = tgfr.get_denorm_data(from_param, to_param)
fuel_report_df = tgfr.create_fuelreport_df(denorm_df)

## OP data
-   op_data_df : read data from OPmovement.xlsx
-   total_flights : summary total of flights in each month

In [8]:
# read excel file
op_data_df = pd.read_excel('./data/OpMovement.xlsx',sheet_name = 'OpMovement', usecols = 'A:H')
op_data_df['Date'] = op_data_df.Date.apply(pd.to_datetime)

# append column month and year
years = []
months = []
for date in op_data_df['Date']:
    years.append(date.year)
    months.append(date.month)

op_data_df['year'] = years
op_data_df['month'] = months
op_data_df['Registration'] = 'HS-' + op_data_df.Registration

total_flights = op_data_df.groupby(['year', 'month', 'Acft Type'])['Flt No'].count()

In [9]:
total_flights

year  month  Acft Type
2020  10     359           95
             33H           69
             77B           66
      11     359          117
             33H           81
                         ... 
2022  5      35A          164
             77B          268
             77E          136
             77W          389
             77Y          118
Name: Flt No, Length: 99, dtype: int64

## eOFP data
-   eofp_df : dataframe contains only eofp data
-   eofp_df_merged : eofp_df merage with data from flight plan database

In [5]:
# initialize number or eOFP report to be downloaded
total_data = len(fuel_report_df)
total_loop = total_data // 500

# load first chunk of eOFP data
eofp_df = tgfr.get_oefp_user_input()

for i in np.arange(total_loop):
    limit = 500
    skip = 500 + 500 * i
    chunk = tgfr.get_oefp_user_input(skip=skip, limit=limit)
    
    # # Don't append last chunk
    # if len(chunk) < 500:
    #     break

    eofp_df = pd.concat([eofp_df, chunk], ignore_index= True)

eofp_df_merged = tgfr.merge_flightPlan_eofp(eofp_df)

## TGFR data
-   tgfr_df: data from Surver.csv. This file is no longer update.

In [6]:
tgfr_df = pd.read_csv('./data/Survey.csv',sep=';')
date_time = []
for date in tgfr_df['flightDate']:
    date_time.append(pd.to_datetime(date))
tgfr_df['flightDate'] = date_time
tgfr_df[['departure', 'arrival']] = tgfr_df['flightSector'].str.split('-', expand=True)
tgfr_columns = ['flightDate', 'flightNumber', 'acRegistration', 'departure', 'arrival',
    'plannedZFW', 'updatedZFW', 'ZFW',
    'plannedFlightTime', 
    'plannedTaxiFuel', 'plannedRampFuel', 'plannedTripFuel',
    'rampFuel','landingFuel','parkingFuel', 
    'offBlock', 'airborne', 'onGround', 'onBlock', 
    'takeOffFuel', 'planningType',
    'pfId', 'pmId', 'apuForParking']
tgfr_df = tgfr_df[tgfr_columns]

## QAR data
- qar_df

In [72]:
# import qar_data from denorm_df
# qar_df = denorm_df.qar.apply(pd.Series)
# qar_cols = ['actual_flight_time',
#             'apu_used_during_taxi_in',
#             'average_cruise_altitude',
#             'climb_duration',
#             'climb_fuel_burn',
#             'cruise_duration',
#             'cruise_fuel_burn',
#             'cruise_distance',
#             'descent_fuel_burn',
#             'gross_weight',
#             'landing_config_height',
#             'landing_flaps',
#             'one_engine_taxi_in',
#             'overall_fuel_used',
#             'pack_off_takeoff',
#             'ramp_fuel',
#             'reduced_flaps_landing',
#             'stabilized_approach_height',
#             'takeoff_flaps',
#             'taxi_in_duration',
#             'taxi_out_fuel',
#             'thrust_reduction_height',
#             'top_of_climb_altitude',
#             'top_of_descent_altitude',
#             'zero_fuel_weight',
#             'auto_land_performed',
#             'idle_reverse_landing_performed',
#             'landing_sector_fuel_burn',
#             'acceleration_height',]
# # import flight date, flight number, dep, arr, aircraft regis from denorm_df
# qar_df = qar_df[qar_cols]
# qar_df[
#     [
#         'flight_date',
#         'flight_number',
#         'departure_aerodrome_icao_code',
#         'aircraft_registration',
#         'aircraft_type'
#         ]
#         ] = denorm_df[
#     [
#         'flight_date',
#         'flight_number',
#         'departure_aerodrome_icao_code',
#         'aircraft_registration',
#         'aircraft_aircraft_config_fleet_name']]

# # create date column
# qar_df['date'] = qar_df.flight_date.apply(pd.to_datetime)
# qar_df.drop(columns= 'flight_date')

# # Remove "-" from aircraft registration
# qar_df['aircraft_registration'] = qar_df['aircraft_registration'].str.replace('-',"")

In [98]:
# import qar from dump database

# formatting float number
pd.options.display.float_format = '{:,.2f}'.format

# load qar to DataFrame
qar_df = pd.read_json('.\data\qar.json')

sel_cols = ['aircraft_registration',
            'qar_start_time',
            'aircraft_type',
            'departure_aerodrome',
            'one_engine_taxi_out',
            'takeoff_flaps',
            'pack_off_takeoff',
            'acceleration_height',
            'climb_sd',
            'descent_sd',
            'landing_config_height',
            'reduced_flaps_landing',
            'idle_reverse_landing_performed',
            'landing_flaps',
            'one_engine_taxi_in',
            'apu_used_during_taxi_in']
qar_df = qar_df[sel_cols]

# convert qar start date to date column
qar_df['date'] = qar_df.qar_start_time.apply(pd.Series)
qar_df['date'] = qar_df.date.apply(pd.to_datetime)
qar_df.drop(columns='qar_start_time', inplace=True)

C:\Users\peaju\AppData\Local\Temp\ipykernel_416\2786985307.py:28: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  qar_df['date'] = qar_df.qar_start_time.apply(pd.Series)
C:\Users\peaju\AppData\Local\Temp\ipykernel_416\2786985307.py:28: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  qar_df['date'] = qar_df.qar_start_time.apply(pd.Series)


In [99]:
# set date column as index
qar_df.set_index(qar_df.date, drop=True, inplace=True)

import qardata

# Change aircraft type between B788 and B789
for i, row in qar_df.iterrows():
    qar_df.loc[i,'aircraft_type'] = qardata.b787_type(row[0], row[1])
    qar_df.loc[i,'is_less_flaps_takeoff'] = qardata.takeoff_flap(row[1], row[4])
    qar_df.loc[i,'is_decel_approach'] = qardata.decel_app(row[9])

# Change column in boolean type to integer value 0/1
qar_df[['one_engine_taxi_out', 'one_engine_taxi_in','is_less_flaps_takeoff','is_decel_approach']] = qar_df[['one_engine_taxi_out', 'one_engine_taxi_in','is_less_flaps_takeoff','is_decel_approach']].astype(int)

# Count flight that didn't not depart from VHHH
qar_df['is_not_hkg']=qar_df.departure_aerodrome != 'VHHH'

# select data for year 2021 until 2050
qar_df.sort_index(ascending=True, inplace=True)
qar_df_2021 = qar_df.loc['2021-01-01':'2050-12-31']

# Remove TJS data in year 2022
qar_df_2021 = qar_df_2021[~((qar_df_2021.date >= "2022-01-01")&(qar_df_2021.aircraft_registration == "HS-TJS"))]

C:\Users\peaju\AppData\Local\Temp\ipykernel_416\3102077592.py:20: FutureWarning: Value based partial slicing on non-monotonic DatetimeIndexes with non-existing keys is deprecated and will raise a KeyError in a future Version.
  qar_df_2021 = qar_df.loc['2021-01-01':'2050-12-31']


In [100]:
# data are summary by sum, average and count.

sum_col = [
    'one_engine_taxi_out',
    'is_less_flaps_takeoff',
    'pack_off_takeoff',
    'is_decel_approach',
    'reduced_flaps_landing',
    'idle_reverse_landing_performed',
    'one_engine_taxi_in',
    'apu_used_during_taxi_in',
    'is_not_hkg'
]
avg_col = [
    'acceleration_height',
    'climb_sd',
    'descent_sd',
]

# Create group by object
groupped_df = qar_df_2021.groupby([qar_df_2021.index.year, qar_df_2021.index.month,qar_df_2021.aircraft_type])

In [101]:
summary_by_sum = groupped_df.sum()[sum_col]
summary_by_avg = groupped_df.mean()[avg_col]
total_qar = groupped_df['aircraft_registration'].count()

# Merge all table and rename count of aircraft registration to total_qar
report = summary_by_sum.join(summary_by_avg)
report = report.join(total_qar).rename(columns={'aircraft_registration':'total_qar'})

#rearrange column to fit report
column_order = [
    'total_qar',
    'is_not_hkg',
    'one_engine_taxi_out',
    'is_less_flaps_takeoff',
    'pack_off_takeoff',
    'acceleration_height',
    'climb_sd',
    'descent_sd',
    'is_decel_approach',
    'reduced_flaps_landing',
    'idle_reverse_landing_performed',
    'one_engine_taxi_in',
    'apu_used_during_taxi_in'
]
report = report[column_order]

#change column name
rename_dict = {
    'total_qar' : 'Total QAR',
    'is_not_hkg' : 'Total Flight exc HKG',
    'one_engine_taxi_out' : 'RETO',
    'is_less_flaps_takeoff' : 'Take off flaps',
    'pack_off_takeoff' : 'Pack Off',
    'acceleration_height' : 'Accel Height',
    'climb_sd' : 'CCO',
    'descent_sd' : 'CDO',
    'is_decel_approach' : 'Decel Appch',
    'reduced_flaps_landing' : 'Landing Flaps',
    'idle_reverse_landing_performed' : 'Idle Rev',
    'one_engine_taxi_in' : 'RETI',
    'apu_used_during_taxi_in' : 'APU start'
}
report.rename(columns= rename_dict, inplace=True)

In [103]:
#Export to csv
report.to_csv('./data/qar_summary.csv')

# Fuel initiative reports

In [106]:
final_df = tgfr.fuel_initiative_data(eofp_df_merged)

AttributeError: module 'tgfr' has no attribute 'fuel_initiative_data'

In [47]:
# fuel initiatives report
import qardata as qar

# na_list = [
#     "average_cruise_altitude",
#     "climb_duration",
#     'climb_fuel_burn',           
#     'cruise_duration',
#     'cruise_fuel_burn', 
#     'cruise_distance',
#     'descent_fuel_burn',
#     'gross_weight',
#     'landing_config_height',
#     'landing_flaps',
#     'one_engine_taxi_in',
#     'overall_fuel_used',
#     'pack_off_takeoff',
#     'reduced_flaps_landing',
#     'stabilized_approach_height',
#     'takeoff_flaps',
#     'taxi_in_duration',
#     'taxi_out_fuel',
#     'thrust_reduction_height',
#     'top_of_climb_altitude',
#     'top_of_descent_altitude',
#     'auto_land_performed',
#     'idle_reverse_landing_performed',
#     'landing_sector_fuel_burn',        
#     'acceleration_height'     
# ]
# qar_df.dropna(subset=[na_list], axis=0)
fuel_initiative_report = qar_df.copy()

# change B787 aircraft type
for i, row in fuel_initiative_report.iterrows():
    fuel_initiative_report.loc[i, 'aircraft_type'] = qar.b787_type(row[32], row[33])
    fuel_initiative_report.loc[i,'is_less_flaps_takeoff'] = qar.takeoff_flap(row[33], row[19])
    fuel_initiative_report.loc[i,'is_decel_approach'] = qar.decel_app(row[10])

In [71]:
fuel_initiative_report.columns

Index(['actual_flight_time', 'apu_used_during_taxi_in',
       'average_cruise_altitude', 'climb_duration', 'climb_fuel_burn',
       'cruise_duration', 'cruise_fuel_burn', 'cruise_distance',
       'descent_fuel_burn', 'gross_weight', 'landing_config_height',
       'landing_flaps', 'one_engine_taxi_in', 'overall_fuel_used',
       'pack_off_takeoff', 'ramp_fuel', 'reduced_flaps_landing',
       'stabilized_approach_height', 'takeoff_flaps', 'taxi_in_duration',
       'taxi_out_fuel', 'thrust_reduction_height', 'top_of_climb_altitude',
       'top_of_descent_altitude', 'zero_fuel_weight', 'auto_land_performed',
       'idle_reverse_landing_performed', 'landing_sector_fuel_burn',
       'acceleration_height', 'flight_date', 'flight_number',
       'departure_aerodrome_icao_code', 'aircraft_registration',
       'aircraft_type', 'date', 'year', 'month', 'is_less_flaps_takeoff',
       'is_decel_approach'],
      dtype='object')

In [70]:
sel_cols = ['aircraft_registration',
            'date',
            'aircraft_type',
            'departure_aerodrome_icao_code',
            'one_engine_taxi_out',
            'takeoff_flaps',
            'pack_off_takeoff',
            'acceleration_height',
            'climb_sd',
            'descent_sd',
            'landing_config_height',
            'reduced_flaps_landing',
            'idle_reverse_landing_performed',
            'landing_flaps',
            'one_engine_taxi_in',
            'apu_used_during_taxi_in']
fuel_initiative_report = fuel_initiative_report[sel_cols]

KeyError: "['departure_aerodrome', 'one_engine_taxi_out', 'climb_sd', 'descent_sd'] not in index"

In [60]:
qar_summary = fuel_initiative_report.groupby(['year','month','aircraft_type'])
qar_summary_exc_hkg = fuel_initiative_report[fuel_initiative_report.departure_aerodrome_icao_code != 'VHHH'].groupby(['year', 'month', 'aircraft_type'])

In [67]:
summary_df = pd.concat([qar_summary['takeoff_flaps'].count(), qar_summary['takeoff_flaps'].count()])

In [68]:
summary_df

year  month  aircraft_type
2022  4      A330               0
             A350             574
             B777             765
             B788             268
             B789              19
      5      A350             618
             B777             756
             B788             125
             B789              33
      4      A330               0
             A350             574
             B777             765
             B788             268
             B789              19
      5      A350             618
             B777             756
             B788             125
             B789              33
Name: takeoff_flaps, dtype: int64

In [18]:
denorm_df[['airborne', 'off_block',
       'on_block', 'on_ground']]

,airborne,off_block,on_block,on_ground
0,NaN,NaN,NaN,NaN
1,2022-05-31T00:40:00.000Z,2022-05-31T00:15:00.000Z,2022-05-31T09:47:00.000Z,2022-05-31T09:38:00.000Z
2,2022-05-31T05:38:00.000Z,2022-05-31T05:28:00.000Z,2022-05-31T07:26:00.000Z,2022-05-31T07:17:00.000Z
3,2022-05-31T04:46:00.000Z,2022-05-31T04:25:00.000Z,2022-05-31T06:48:00.000Z,2022-05-31T06:37:00.000Z
4,2022-05-31T02:03:00.000Z,2022-05-31T01:49:00.000Z,2022-05-31T10:20:00.000Z,2022-05-31T10:11:00.000Z
...,...,...,...,...
3853,2022-04-01T19:07:00.000Z,2022-04-01T18:25:00.000Z,2022-04-02T06:39:00.000Z,1901-01-01T00:00:00.000Z
3854,2022-04-01T19:53:00.000Z,2022-04-01T19:02:00.000Z,2022-04-02T06:45:00.000Z,1901-01-01T00:00:00.000Z
3855,2022-04-01T19:00:00.000Z,2022-04-01T18:35:00.000Z,2022-04-02T07:11:00.000Z,1901-01-01T00:00:00.000Z
3856,2022-04-01T02:45:00.000Z,2022-04-01T02:25:00.000Z,2022-04-01T03:53:00.000Z,2022-04-01T03:49:00.000Z


In [109]:
fuel_initiative_report[['aircraft_registration', 'aircraft_type']].sample(20)

,aircraft_registration,aircraft_type
3039,HS-TKW,B777
348,HS-TKZ,B777
2674,HS-TKO,B777
1049,HS-THM,A350
1555,HS-TKL,B777
541,HS-THL,A350
2309,HS-TKK,B777
634,HS-TQD,B788
676,HS-TJW,B777
164,HS-THL,A350
